In [2]:
from od3d.cv.io import read_image
from od3d.cv.visual.show import show_img, show_imgs
from od3d.cv.visual.crop import crop
import torch
from pathlib import Path
from od3d.cv.geometry.fit.phase_corr import get_corr_amax, polar_transformer, logpolar_filter, get_affine_from_imgs
import logging
from logging import getLogger
logging.basicConfig(level=logging.INFO)
logger = getLogger(__name__)


device = 'cuda:0'
img_a = 1 - read_image(Path('nine.png')) / 255. 
img_a = img_a[:3].to(device=device)
H, W = img_a.shape[-2:]

scale = 1.
angle = 100.
transl_x = 0 #-10
transl_y = 0 # -30

from kornia.geometry.transform import warp_affine, get_affine_matrix2d # (src, M
# M = Bx2x3
M = torch.zeros((2, 3))
scale = torch.Tensor([scale, scale]).to(device)[None,]
angle = torch.Tensor([angle,]).to(device)
transl = torch.Tensor([transl_x, transl_y]).to(device)[None,]
center = torch.Tensor([H/2, W/2]).to(device)[None,]
M = get_affine_matrix2d(translations=transl, angle=angle, scale=scale, center=center)
img_b = warp_affine(img_a[None,], M=M[:, :2, :3], dsize=(H, W))[0]

#center = torch.Tensor([(W / 2.) + transl_x, (H / 2.) + transl_y]).to(device)
#img_b, cam_crop_tform_cam = crop(img_a, H_out=H, W_out=W, center=center)
img_ab = torch.stack([img_a, img_b], dim=0)
#print(img_ab.shape)

img_b_polar, _ = polar_transformer(img_b[..., None], out_size=(H, W), device=device)
img_b_polar = img_b_polar.permute(3, 0, 1, 2)[0]

img_ab = torch.stack([img_a, img_b, img_b_polar], dim=0)

# show_imgs(img_ab)

from od3d.cv.geometry.fit.phase_corr import get_corr_amax, polar_transformer, logpolar_filter, get_affine_from_imgs

M_est = get_affine_from_imgs(img_a=img_a, img_b=img_b)

logger.info(M)
logger.info(M_est)

# 130 -> -50
# 45 -> 45
# 20 -> 20
# -20 -> -20
# -45 -> -45
# -130 -> 50

INFO:od3d.cv.geometry.fit.phase_corr:angle tensor([ 98.8235, 278.8235], device='cuda:0'), scale tensor([[0.9808, 0.9808],
        [0.9808, 0.9808]], device='cuda:0')
INFO:od3d.cv.geometry.fit.phase_corr:transl_x 2, transl_y 3
INFO:__main__:tensor([[[-1.7365e-01, -9.8481e-01,  2.7520e+02],
         [ 9.8481e-01, -1.7365e-01,  2.4077e+01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00]]], device='cuda:0')
INFO:__main__:tensor([[-1.5045e-01, -9.6919e-01,  2.7225e+02],
        [ 9.6919e-01, -1.5045e-01,  2.6110e+01],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0')


In [3]:
M_est = get_affine_from_imgs(img_a=img_a, img_b=img_b)

logger.info(M)
logger.info(M_est)

INFO:od3d.cv.geometry.fit.phase_corr:angle tensor([ 98.8235, 278.8235], device='cuda:0'), scale tensor([[0.9808, 0.9808],
        [0.9808, 0.9808]], device='cuda:0')
INFO:od3d.cv.geometry.fit.phase_corr:transl_x 2, transl_y 3
INFO:__main__:tensor([[[-1.7365e-01, -9.8481e-01,  2.7520e+02],
         [ 9.8481e-01, -1.7365e-01,  2.4077e+01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00]]], device='cuda:0')
INFO:__main__:tensor([[-1.5045e-01, -9.6919e-01,  2.7225e+02],
        [ 9.6919e-01, -1.5045e-01,  2.6110e+01],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0')


In [4]:
M

tensor([[[ -0.6428,  -0.7660, 307.1261],
         [  0.7660,  -0.6428, 111.7848],
         [  0.0000,   0.0000,   1.0000]]], device='cuda:0')

In [5]:
get_affine_from_imgs(img_a=img_a, img_b=img_b)


tensor([[ 0.4775, -0.1716, 98.4964],
        [ 0.1716,  0.4775, 74.7455],
        [ 0.0000,  0.0000,  1.0000]], device='cuda:0')

In [3]:
get_affine_from_imgs(img_a=img_a, img_b=img_b)


tensor([[ 0.4704, -0.1625, 99.2482],
        [ 0.1625,  0.4704, 77.8117],
        [ 0.0000,  0.0000,  1.0000]], device='cuda:0')

In [4]:
import kornia.geometry as KG

registrator = KG.ImageRegistrator('similarity').to(device=device)
model, interm = registrator.register(img_a[None,], img_b[None,], output_intermediate_models=True)
model

tensor([[[ 1.1363, -0.0031,  0.0904],
         [ 0.0031,  1.1363, -0.1098],
         [ 0.0000,  0.0000,  1.0000]]], device='cuda:0', grad_fn=<CopySlices>)

In [3]:


h = logpolar_filter((H, W)).to(device=device) #highpass((source.shape[1],source.shape[2])) # [H,W]

img_a_fft = torch.view_as_real(torch.fft.fft2(img_a))
img_a_fft = torch.sqrt(img_a_fft[..., 0]**2 + img_a_fft[..., 1]**2 + 1e-15)
img_a_fft  = torch.fft.fftshift(img_a_fft, dim=(-2, -1))
img_a_fft *= h[None,]

img_b_fft = torch.view_as_real(torch.fft.fft2(img_b))
img_b_fft = torch.sqrt(img_b_fft[..., 0]**2 + img_b_fft[..., 1]**2 + 1e-15)
img_b_fft  = torch.fft.fftshift(img_b_fft, dim=(-2, -1))
img_b_fft *= h[None,]

img_a_fft_polar, img_a_fft_polar_logbase = polar_transformer(img_a_fft[..., None], out_size=(H, W), device=device)
img_a_fft_polar = img_a_fft_polar.permute(3, 0, 1, 2)[0]

img_b_fft_polar, img_b_fft_polar_logbase = polar_transformer(img_b_fft[..., None], out_size=(H, W), device=device)
img_b_fft_polar = img_b_fft_polar.permute(3, 0, 1, 2)[0]

index_corr_max_x, index_corr_max_y = get_corr_amax(img_a_fft_polar, img_b_fft_polar)

angle = index_corr_max_y / (H) * 180
scale = torch.pow(img_b_fft_polar_logbase, index_corr_max_x.float())

scale = torch.Tensor([scale, scale]).to(device)[None,]
angle = torch.Tensor([angle,]).to(device)
transl = torch.Tensor([0, 0]).to(device)[None,]
center = torch.Tensor([W/2, H/2]).to(device)[None,]
M = get_affine_matrix2d(translations=transl, angle=angle, scale=scale, center=center)
img_a_rot_scaled = warp_affine(img_a[None,], M=M[:, :2, :3], dsize=(H, W))[0]


transl_x, transl_y = get_corr_amax(img_a_rot_scaled, img_b)
transl = torch.Tensor([-transl_x, -transl_y]).to(device)[None,]
M = get_affine_matrix2d(translations=transl, angle=angle, scale=scale, center=center)

print(scale, angle, transl_x, transl_y)


tensor([[0.5074, 0.5074]], device='cuda:0') tensor([19.7647], device='cuda:0') tensor(-10, device='cuda:0') tensor(-30, device='cuda:0')


In [9]:
img_b_fft_polar_logbase

NameError: name 'img_b_fft_polar_logbase' is not defined

(tensor(0., device='cuda:0'), tensor(1., device='cuda:0', dtype=torch.float64))

In [10]:
img_a_fft.shape

torch.Size([3, 255, 255])

(tensor(-10, device='cuda:0'), tensor(-30, device='cuda:0'))

In [ ]:
z = torch.polar(abs, angle)
